In [33]:
import os
import json

import pandas as pd
import numpy as np
from tqdm import tqdm
from copy import deepcopy


In [34]:
file_name = 'result_yolov5_yolov7_swin_all_wopseudo_wbfensemble.csv'
file_dir = '/opt/ml/' + file_name
pseudo = pd.read_csv(file_dir)
pseudo

,PredictionString,image_id
0,7 0.8889930248260498 216.75791931152344 51.973...,test/0000.jpg
1,5 0.6938202381134033 132.2281494140625 0.22078...,test/0001.jpg
2,1 0.8105384707450867 76.7972183227539 272.6375...,test/0002.jpg
3,9 0.7210465669631958 144.7139434814453 259.471...,test/0003.jpg
4,1 0.6025466124216715 195.54220581054688 255.07...,test/0004.jpg
...,...,...
4866,1 0.5896831671396892 293.3543701171875 348.683...,test/4866.jpg
4867,3 0.8107656240463257 414.9770812988281 319.381...,test/4867.jpg
4868,7 0.8208968639373779 115.30994415283203 453.55...,test/4868.jpg
4869,1 0.6532298922538757 842.8405151367188 0.37100...,test/4869.jpg


In [35]:
data_dir = '/opt/ml/dataset'
test_json = json.load(open(os.path.join(data_dir,'test.json')))

In [36]:
pseudo_json = deepcopy(test_json)
pseudo_json['images'] = []

In [37]:
pseudo_json['images']

[]

In [ ]:
thr = 0.6

box_id=0
for i in tqdm(range(4871)):

    if type(pseudo['PredictionString'][i])==str: # pred가 없는 이미지는 무시
        
        for j in range(len(pseudo['PredictionString'][i].split(' '))//6):
            o = pseudo['PredictionString'][i].split(' ')
            id, score, xmin, ymin, xmax, ymax = map(float,o[6*j: 6*(j+1)])
            
            if score<thr:
                continue

            id, xmin, ymin, xmax, ymax = map(int, [id, xmin, ymin, xmax, ymax])
            w = xmax - xmin
            h = ymax - ymin
            
            info = dict()
            info['image_id'] = i
            info['category_id'] = id
            info['area'] = w*h
            info['bbox'] = [xmin, ymin, w, h]
            info['iscrowd'] = 0 # 겹친 박스가 있는지 여부
            info['id'] = box_id # 박스 id
            box_id += 1
            pseudo_json['annotations'].append(info)
            pseudo_json['images'].append(test_json['images'][i])

image_list = []
for i in set([i['id'] for i in pseudo_json['images']]):
    image_list.append(test_json['images'][i])
pseudo_json['images'] = image_list

In [55]:
with open(os.path.join(data_dir, f"pseudo_final.json"),'w') as f:
        json.dump(pseudo_json, f, indent=4)